<a href="https://colab.research.google.com/github/Miguel72-bit/CORTE2/blob/main/LuisMiguelOrdonezCorte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bitsandbytes accelerate transformers
!pip install gradio peft transformers accelerate bitsandbytes
!pip install transformers torch

  Using cached gradio-5.29.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.10.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached ruff-0.11.9-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
Using cached gradio-5.29.0-py3-none-any.whl (54.1 MB)
Using cached gradio_client-1.10.0-py3-none-any.whl (322 kB)
Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
Using cached fastapi-0.115.12-py3-none-any.whl (95 kB)
Us

# Codigo 1


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# Configurar las variables de entorno para la caché de modelos
# Establecer la carpeta donde se almacenarán los modelos descargados
os.environ["TRANSFORMERS_CACHE"] = "./model_cache"
def cargar_modelo(nombre_modelo):
    """
    Carga un modelo pre-entrenado y su tokenizador correspondiente.

    Args:
        nombre_modelo (str): Identificador del modelo en Hugging Face Hub

    Returns:
        tuple: (modelo, tokenizador)
    """
    # Cargar el modelo y el tokenizador desde Hugging Face
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)

    # Configurar el modelo para inferencia
    modelo = modelo.eval()  # Ponemos el modelo en modo evaluación (sin entrenamiento)

    # Si hay GPU disponible, configuramos el modelo para usarla
    dispositivo = verificar_dispositivo()
    modelo = modelo.to(dispositivo)

    return modelo, tokenizador

def verificar_dispositivo():
    """
    Verifica el dispositivo disponible (CPU/GPU) y muestra información relevante.

    Returns:
        torch.device: Dispositivo a utilizar
    """
    # Comprobar si hay GPU disponible
    if torch.cuda.is_available():
        dispositivo = torch.device("cuda")  # Usar GPU si está disponible
        print(f"✅ Usando GPU: {torch.cuda.get_device_name(0)}")
    else:
        dispositivo = torch.device("cpu")  # Si no, usar CPU
        print("⚠️ Usando CPU (GPU no disponible)")

    return dispositivo

# Función principal de prueba
def main():
    dispositivo = verificar_dispositivo()
    print(f"Utilizando dispositivo: {dispositivo}")

    # Cargar un modelo pequeño adecuado para chatbots, por ejemplo, GPT-2
    modelo, tokenizador = cargar_modelo("gpt2")  # Puedes reemplazar "gpt2" por otro modelo

    # Realizar una prueba simple de generación de texto
    entrada = "What color comes out of blue and yellow??"
    inputs = tokenizador(entrada, return_tensors="pt").to(dispositivo)

    # Generar una respuesta usando el modelo
    with torch.no_grad():
        salida = modelo.generate(inputs["input_ids"], max_length=50, num_return_sequences=1)

    # Decodificar la salida generada
    respuesta = tokenizador.decode(salida[0], skip_special_tokens=True)

    print(f"Entrada: {entrada}")
    print(f"Respuesta generada: {respuesta}")

if __name__ == "__main__":
    main()



✅ Usando GPU: Tesla T4
Utilizando dispositivo: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Usando GPU: Tesla T4


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Entrada: What color comes out of blue and yellow??
Respuesta generada: What color comes out of blue and yellow??

I'm not sure if it's a color or a color. I'm not sure if it's a color or a color.

I'm not sure if it's a color or a


# Codigo 2

In [24]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Configuración del modelo y tokenizador
def cargar_modelo(nombre_modelo):
    """
    Carga un modelo pre-entrenado y su tokenizador correspondiente.

    Args:
        nombre_modelo (str): Identificador del modelo en Hugging Face Hub

    Returns:
        tuple: (modelo, tokenizador)
    """
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)

    # Asignar el token de padding si no tiene
    if tokenizador.pad_token is None:
        tokenizador.pad_token = tokenizador.eos_token  # Usamos el token EOS como PAD

    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    modelo = modelo.to(dispositivo)  # Mover el modelo a la GPU si está disponible

    return modelo, tokenizador, dispositivo

def preprocesar_entrada(texto, tokenizador, longitud_maxima=512, dispositivo=None):
    """
    Preprocesa el texto de entrada para pasarlo al modelo.

    Args:
        texto (str): Texto de entrada del usuario
        tokenizador: Tokenizador del modelo
        longitud_maxima (int): Longitud máxima de la secuencia
        dispositivo (torch.device): Dispositivo donde ejecutar el modelo

    Returns:
        torch.Tensor: Tensor de entrada para el modelo
    """
    # Añadir tokens especiales y convertir a tensor
    tokens = tokenizador.encode(texto, truncation=True, padding="max_length", max_length=longitud_maxima, return_tensors="pt")

    # Pasar los tokens al dispositivo adecuado (GPU/CPU)
    tokens = tokens.to(dispositivo)

    return tokens

def generar_respuesta(modelo, entrada_procesada, tokenizador, parametros_generacion=None):
    """
    Genera una respuesta basada en la entrada procesada.

    Args:
        modelo: Modelo de lenguaje
        entrada_procesada: Tokens de entrada procesados
        tokenizador: Tokenizador del modelo
        parametros_generacion (dict): Parámetros para controlar la generación

    Returns:
        str: Respuesta generada
    """
    if parametros_generacion is None:
        parametros_generacion = {
            "do_sample": True,  # Activar el muestreo para usar temperature y top_p
            "max_new_tokens": 50,  # Generar hasta 50 tokens nuevos
            "temperature": 0.7,  # Controla la aleatoriedad
            "top_p": 0.9,  # Controla el muestreo (top-p sampling)
            "top_k": 50,  # Número de palabras candidatas a considerar
        }

    # Generar la respuesta usando el modelo
    with torch.no_grad():
        salida = modelo.generate(entrada_procesada, **parametros_generacion)

    # Decodificar la salida generada
    respuesta = tokenizador.decode(salida[0], skip_special_tokens=True)

    return respuesta

def crear_prompt_sistema(instrucciones):
    """
    Crea un prompt de sistema para dar instrucciones al modelo.

    Args:
        instrucciones (str): Instrucciones sobre cómo debe comportarse el chatbot

    Returns:
        str: Prompt formateado
    """
    return f"El siguiente es un chatbot. Instrucciones: {instrucciones}\n"

# Ejemplo de uso
def interaccion_simple():
    modelo, tokenizador, dispositivo = cargar_modelo("gpt2")

    # Crear un prompt de sistema para definir la personalidad del chatbot
    instrucciones = "El chatbot debe ser amistoso, educado y brindar respuestas claras y concisas."
    prompt_sistema = crear_prompt_sistema(instrucciones)

    # Procesa una entrada de ejemplo
    entrada_usuario = "¿What color comes out of red and yellow??"
    entrada_procesada = preprocesar_entrada(prompt_sistema + entrada_usuario, tokenizador, dispositivo=dispositivo)

    # Genera y mostrar la respuesta
    respuesta = generar_respuesta(modelo, entrada_procesada, tokenizador)
    print(f"Respuesta del chatbot: {respuesta}")

if __name__ == "__main__":
    interaccion_simple()




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Respuesta del chatbot: El siguiente es un chatbot. Instrucciones: El chatbot debe ser amistoso, educado y brindar respuestas claras y concisas.
¿What color comes out of red and yellow??The new version of the U.S. Constitution, U.S. Code, is the most comprehensive and comprehensive of the U.S. Constitution. It also provides a blueprint for how to live and work in the United States.

It


# Codigo 3

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def cargar_modelo(nombre_modelo):
    """
    Carga un modelo pre-entrenado y su tokenizador correspondiente.

    Args:
        nombre_modelo (str): Identificador del modelo en Hugging Face Hub

    Returns:
        tuple: (modelo, tokenizador, dispositivo)
    """
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)


    if tokenizador.pad_token is None:
        tokenizador.pad_token = tokenizador.eos_token  # Usamos el token EOS como PAD

    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    modelo = modelo.to(dispositivo)  # Mover el modelo a la GPU si está disponible

    return modelo, tokenizador, dispositivo  # Devolver tres valores

class GestorContexto:
    """
    Clase para gestionar el contexto de una conversación con el chatbot.
    """

    def __init__(self, longitud_maxima=1024, formato_mensaje=None):
        self.historial = []
        self.longitud_maxima = longitud_maxima
        self.formato_mensaje = formato_mensaje or self._formato_predeterminado

    def _formato_predeterminado(self, rol, contenido):
        return f"{rol}: {contenido}"

    def agregar_mensaje(self, rol, contenido):
        mensaje_formateado = self.formato_mensaje(rol, contenido)
        self.historial.append(mensaje_formateado)

    def construir_prompt_completo(self):
        return "\n".join(self.historial)

    def truncar_historial(self, tokenizador):
        while True:
            tokens = tokenizador.encode(self.construir_prompt_completo(), return_tensors="pt")
            if len(tokens[0]) > self.longitud_maxima:
                self.historial.pop(0)
            else:
                break

class Chatbot:
    """
    Implementación de chatbot con manejo de contexto.
    """

    def __init__(self, modelo_id, instrucciones_sistema=None):
        """
        Inicializa el chatbot.

        Args:
            modelo_id (str): Identificador del modelo en Hugging Face
            instrucciones_sistema (str): Instrucciones de comportamiento del sistema
        """
        # Desempaquetar los tres valores: modelo, tokenizador, y dispositivo
        self.modelo, self.tokenizador, self.dispositivo = cargar_modelo(modelo_id)
        self.gestor_contexto = GestorContexto()

        # Inicializar el contexto con instrucciones del sistema
        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje('sistema', instrucciones_sistema)

    def responder(self, mensaje_usuario, parametros_generacion=None):
        """
        Genera una respuesta al mensaje del usuario.

        Args:
            mensaje_usuario (str): Mensaje del usuario
            parametros_generacion (dict): Parámetros para la generación

        Returns:
            str: Respuesta generada
        """
        # 1. Agregar mensaje del usuario al contexto
        self.gestor_contexto.agregar_mensaje('usuario', mensaje_usuario)

        # 2. Construir el prompt completo
        prompt_completo = self.gestor_contexto.construir_prompt_completo()

        # 3. Preprocesar la entrada
        entrada_procesada = preprocesar_entrada(prompt_completo, self.tokenizador, dispositivo=self.dispositivo)

        # 4. Generar la respuesta
        respuesta = generar_respuesta(self.modelo, entrada_procesada, self.tokenizador, parametros_generacion)

        # 5. Agregar respuesta al contexto
        self.gestor_contexto.agregar_mensaje('asistente', respuesta)

        # 6. Devolver la respuesta
        return respuesta

# Funciones auxiliares
def preprocesar_entrada(texto, tokenizador, longitud_maxima=512, dispositivo=None):
    tokens = tokenizador.encode(texto, truncation=True, padding="max_length", max_length=longitud_maxima, return_tensors="pt")
    tokens = tokens.to(dispositivo)
    return tokens

def generar_respuesta(modelo, entrada_procesada, tokenizador, parametros_generacion=None):
    if parametros_generacion is None:
        parametros_generacion = {
            "do_sample": True,
            "max_new_tokens": 50,
            "temperature": 0.7,
            "top_p": 0.9,
            "top_k": 50,
        }

    with torch.no_grad():
        salida = modelo.generate(entrada_procesada, **parametros_generacion)

    respuesta = tokenizador.decode(salida[0], skip_special_tokens=True)
    return respuesta

# Prueba del sistema
def prueba_conversacion():
    instrucciones = "El chatbot debe ser amigable, siempre responde de manera educada y concisa."
    chatbot = Chatbot(modelo_id="gpt2", instrucciones_sistema=instrucciones)

    print(chatbot.responder("¿que mescla sale de rojo y amarillo?"))
    print(chatbot.responder("¿Qué puedes hacer por mí?"))
    print(chatbot.responder("¿quien descubrio america?"))

if __name__ == "__main__":
    prueba_conversacion()




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


sistema: El chatbot debe ser amigable, siempre responde de manera educada y concisa.
usuario: ¿que mescla sale de rojo y amarillo?The most recent example of a new invention that could help the development of robots is a new robotic arm that could be used to help with the development of robots.

The project is called the "Robot Robotic Arm" and has been in


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


sistema: El chatbot debe ser amigable, siempre responde de manera educada y concisa.
usuario: ¿que mescla sale de rojo y amarillo?
asistente: sistema: El chatbot debe ser amigable, siempre responde de manera educada y concisa.
usuario: ¿que mescla sale de rojo y amarillo?The most recent example of a new invention that could help the development of robots is a new robotic arm that could be used to help with the development of robots.

The project is called the "Robot Robotic Arm" and has been in
usuario: ¿Qué puedes hacer por mí?There are more than a few things you can do to help your family and friends get over the stresses of a job.

A common problem for many workers is that they don't have a family home and don't have a car.


sistema: El chatbot debe ser amigable, siempre responde de manera educada y concisa.
usuario: ¿que mescla sale de rojo y amarillo?
asistente: sistema: El chatbot debe ser amigable, siempre responde de manera educada y concisa.
usuario: ¿que mescla sale de rojo 

# Codigo 4

In [25]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import torch.nn as nn

#  importar bitsandbytes y configurar cuantización
try:
    from transformers import BitsAndBytesConfig
    bitsandbytes_available = True
    print("`bitsandbytes` está instalado.")
except ImportError:
    bitsandbytes_available = False
    print("`bitsandbytes` no está instalado, se omitirá la cuantización.")

def configurar_cuantizacion(bits=4):
    """
    Configura los parámetros para la cuantización del modelo.

    Args:
        bits (int): Bits para cuantización (4 u 8)

    Returns:
        BitsAndBytesConfig: Configuración de cuantización
    """
    if bitsandbytes_available:
        # Configuración para 4 bits o 8 bits de cuantización
        config_cuantizacion = BitsAndBytesConfig(
            load_in_4bit=(bits == 4),
            load_in_8bit=(bits == 8)
        )
        return config_cuantizacion
    else:
        print("Cuantización no disponible sin `bitsandbytes`. Se continuará sin cuantización.")
        return None

def cargar_modelo_optimizado(nombre_modelo, optimizaciones=None):
    """
    Carga un modelo con optimizaciones aplicadas.

    Args:
        nombre_modelo (str): Identificador del modelo
        optimizaciones (dict): Diccionario con flags para las optimizaciones

    Returns:
        tuple: (modelo, tokenizador)
    """
    if optimizaciones is None:
        optimizaciones = {
            "cuantizacion": True,
            "bits": 4,
            "offload_cpu": False,
            "flash_attention": True
        }

    # Aplicar cuantización si está habilitada
    if optimizaciones.get("cuantizacion", False) and bitsandbytes_available:
        config_cuantizacion = configurar_cuantizacion(optimizaciones["bits"])
        modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo, quantization_config=config_cuantizacion)
    else:
        modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)

    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)


    # Aquí solo cargamos el modelo con cuantización por simplicidad

    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    modelo.to(dispositivo)

    return modelo, tokenizador, dispositivo

def aplicar_sliding_window(modelo, window_size=1024):
    """
    Configura la atención de ventana deslizante para procesar secuencias largas.

    Args:
        modelo: Modelo a configurar
        window_size (int): Tamaño de la ventana de atención
    """
    # Aquí aplicamos una función de atención deslizante para que el modelo maneje secuencias largas
    # Configurar una ventana deslizante en la capa de atención
    for layer in modelo.transformer.h:
        layer.attn.config.attn_window_size = window_size
    print(f"Ventana de atención deslizante configurada con tamaño: {window_size}")

def evaluar_rendimiento(modelo, tokenizador, texto_prueba, dispositivo):
    """
    Evalúa el rendimiento del modelo en términos de velocidad y memoria.

    Args:
        modelo: Modelo a evaluar
        tokenizador: Tokenizador del modelo
        texto_prueba (str): Texto para pruebas de rendimiento
        dispositivo: Dispositivo donde se ejecutará

    Returns:
        dict: Métricas de rendimiento
    """
    # Preprocesar la entrada
    inputs = tokenizador(texto_prueba, return_tensors="pt").to(dispositivo)

    # Medir el tiempo de inferencia
    inicio = time.time()
    with torch.no_grad():
        modelo(inputs["input_ids"])
    fin = time.time()

    tiempo_inferencia = fin - inicio

    # Uso de memoria
    memoria_MB = torch.cuda.memory_allocated() / (1024 ** 2) if torch.cuda.is_available() else 0

    # Métricas adicionales (tokens por segundo)
    tokens_por_segundo = len(inputs["input_ids"]) / tiempo_inferencia if tiempo_inferencia > 0 else 0

    # Retornar las métricas
    return {
        "tiempo_inferencia": tiempo_inferencia,
        "memoria_MB": memoria_MB,
        "tokens_por_segundo": tokens_por_segundo
    }

# Función de demostración
def demo_optimizaciones():
    # Crear y evaluar diferentes configuraciones
    modelo_base, tokenizador, dispositivo = cargar_modelo_optimizado("gpt2", optimizaciones={"cuantizacion": False})
    modelo_4bits, _, _ = cargar_modelo_optimizado("gpt2", optimizaciones={"cuantizacion": True, "bits": 4})
    modelo_sliding_window, _, _ = cargar_modelo_optimizado("gpt2", optimizaciones={"cuantizacion": False})

    aplicar_sliding_window(modelo_sliding_window)

    # Texto de prueba para evaluar
    texto_prueba = "Hola, ¿cómo estás? Esta es una prueba para evaluar el rendimiento del modelo."

    # Evaluación del rendimiento con el modelo base
    print("Evaluación del modelo base:")
    metricas_base = evaluar_rendimiento(modelo_base, tokenizador, texto_prueba, dispositivo)
    print(metricas_base)

    # Evaluación con el modelo con cuantización de 4 bits
    print("Evaluación con el modelo con cuantización de 4 bits:")
    metricas_4bits = evaluar_rendimiento(modelo_4bits, tokenizador, texto_prueba, dispositivo)
    print(metricas_4bits)

    # Evaluación con el modelo con ventana deslizante
    print("Evaluación con el modelo con ventana deslizante:")
    metricas_sliding_window = evaluar_rendimiento(modelo_sliding_window, tokenizador, texto_prueba, dispositivo)
    print(metricas_sliding_window)

    # Comparar y mostrar las métricas de rendimiento
    print("Comparación de métricas de rendimiento:")
    print(f"Modelo base vs 4 bits: {metricas_base['tiempo_inferencia']} segundos vs {metricas_4bits['tiempo_inferencia']} segundos")
    print(f"Modelo base vs Sliding Window: {metricas_base['tiempo_inferencia']} segundos vs {metricas_sliding_window['tiempo_inferencia']} segundos")

if __name__ == "__main__":
    demo_optimizaciones()


`bitsandbytes` está instalado.
Ventana de atención deslizante configurada con tamaño: 1024
Evaluación del modelo base:
{'tiempo_inferencia': 0.012219905853271484, 'memoria_MB': 5805.74560546875, 'tokens_por_segundo': 81.83369102899286}
Evaluación con el modelo con cuantización de 4 bits:
{'tiempo_inferencia': 0.023812532424926758, 'memoria_MB': 5805.74560546875, 'tokens_por_segundo': 41.994693472971754}
Evaluación con el modelo con ventana deslizante:
{'tiempo_inferencia': 0.010013580322265625, 'memoria_MB': 5805.74560546875, 'tokens_por_segundo': 99.86438095238096}
Comparación de métricas de rendimiento:
Modelo base vs 4 bits: 0.012219905853271484 segundos vs 0.023812532424926758 segundos
Modelo base vs Sliding Window: 0.012219905853271484 segundos vs 0.010013580322265625 segundos


# Codigo 5

In [20]:
!pip install gradio transformers torch



In [23]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Clase para manejar el contexto de la conversación
class GestorContexto:
    def __init__(self, longitud_maxima=1024, formato_mensaje=None):
        self.historial = []
        self.longitud_maxima = longitud_maxima
        self.formato_mensaje = formato_mensaje or self._formato_predeterminado

    def _formato_predeterminado(self, rol, contenido):
        return f"{rol}: {contenido}"

    def agregar_mensaje(self, rol, contenido):
        mensaje_formateado = self.formato_mensaje(rol, contenido)
        self.historial.append(mensaje_formateado)

    def construir_prompt_completo(self):
        # Solo mantenemos los últimos mensajes que sean relevantes
        return "\n".join(self.historial[-5:])  # Tomamos solo los últimos 5 mensajes

    def truncar_historial(self, tokenizador):
        while True:
            tokens = tokenizador.encode(self.construir_prompt_completo(), return_tensors="pt")
            if len(tokens[0]) > self.longitud_maxima:
                self.historial.pop(0)
            else:
                break

# Función para cargar el modelo optimizado (cuantización opcional)
def cargar_modelo_optimizado(nombre_modelo):
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)

    # Verificar si el tokenizador tiene un pad_token, si no, usar el eos_token como pad_token
    if tokenizador.pad_token is None:
        tokenizador.pad_token = tokenizador.eos_token  # Usamos el token EOS como PAD

    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    modelo.to(dispositivo)
    return modelo, tokenizador, dispositivo

# Preprocesamiento de la entrada
def preprocesar_entrada(texto, tokenizador, longitud_maxima=512, dispositivo=None):
    tokens = tokenizador.encode(texto, truncation=True, padding="max_length", max_length=longitud_maxima, return_tensors="pt")
    tokens = tokens.to(dispositivo)
    return tokens

# Generación de la respuesta
def generar_respuesta(modelo, entrada_procesada, tokenizador, parametros_generacion=None):
    if parametros_generacion is None:
        parametros_generacion = {
            "do_sample": True,
            "max_new_tokens": 50,
            "temperature": 0.7,
            "top_p": 0.9,
            "top_k": 50,
        }

    with torch.no_grad():
        salida = modelo.generate(entrada_procesada, **parametros_generacion)

    respuesta = tokenizador.decode(salida[0], skip_special_tokens=True)
    return respuesta

# Clase del chatbot
class Chatbot:
    def __init__(self, modelo_id, instrucciones_sistema=None):
        self.modelo, self.tokenizador, self.dispositivo = cargar_modelo_optimizado(modelo_id)
        self.gestor_contexto = GestorContexto()

        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje('sistema', instrucciones_sistema)

    def responder(self, mensaje_usuario):
        self.gestor_contexto.agregar_mensaje('usuario', mensaje_usuario)
        prompt_completo = self.gestor_contexto.construir_prompt_completo()
        entrada_procesada = preprocesar_entrada(prompt_completo, self.tokenizador, dispositivo=self.dispositivo)
        respuesta = generar_respuesta(self.modelo, entrada_procesada, self.tokenizador)
        self.gestor_contexto.agregar_mensaje('asistente', respuesta)
        return respuesta

# Función para crear la interfaz web con Gradio
def crear_interfaz_web(chatbot):
    """
    Crea una interfaz web simple para el chatbot usando Gradio.

    Args:
        chatbot: Instancia del chatbot

    Returns:
        gr.Interface: Interfaz de Gradio
    """
    # Función de callback para procesar entradas y generar respuestas
    def procesar_entrada(mensaje_usuario):
        return chatbot.responder(mensaje_usuario)

    # Definir los componentes de la interfaz
    interfaz = gr.Interface(
        fn=procesar_entrada,  # Función de procesamiento
        inputs=gr.Textbox(lines=2, placeholder="Escribe tu mensaje aquí..."),  # Entrada de texto
        outputs=gr.Textbox(),  # Salida de texto
        title="Chatbot Personalizado",  # Título de la interfaz
        description="Chatbot que responde preguntas y mantiene el contexto de la conversación."  # Descripción
    )

    return interfaz

# Función principal para el despliegue
def main_despliegue():
    # Cargar el modelo personalizado
    modelo_id = "gpt2"  # O usar tu modelo personalizado si lo tienes
    instrucciones_sistema = "El chatbot debe ser amigable, educado y conciso."

    # Crear instancia del chatbot
    chatbot = Chatbot(modelo_id, instrucciones_sistema)

    # Crear y lanzar la interfaz web
    interfaz = crear_interfaz_web(chatbot)
    interfaz.launch(share=True)  # share=True para obtener un enlace público de la interfaz

if __name__ == "__main__":
    main_despliegue()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d8f2b3780c77072fb6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
